# Download data

You can compose queries to download raw data.

In [ ]:
import getpass
import platform

import matplotlib.pyplot as plt
import pandas as pd

import doplaydo.dodata as dd

username = getpass.getuser()

PROJECT_ID = f"rings-{username}-{platform.node()}"

In [ ]:
with dd.get_session() as session:
    device_data, df = dd.get_data_by_query(
        [dd.Project.project_id == PROJECT_ID], limit=1, session=session
    )[0]
    device_data_pkey = device_data.pkey
    cell_id = device_data.device.cell.cell_id
    device_id = device_data.device.device_id

In [ ]:
device_data_pkey

In [ ]:
cell_id

In [ ]:
device_id

In [ ]:
with dd.get_session() as session:
    device_data20, df20 = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.attribute_filter(dd.Cell, "radius_um", 20),
        ],
        limit=1,
        session=session,
    )[0]
    device_pkey = device_data20.device.pkey
    device_id = device_data20.device.device_id
device_pkey

In [ ]:
device_id

You have access to:

- dd.Project
- dd.Die
- dd.Wafer
- dd.ParentCell
- dd.Cell
- dd.Device
- dd.DeviceData

Where each model has its table columns as attributes.


![](https://i.imgur.com/CfOHl57.png)

You can use `get_data_by_query` to query a subset of data filtered by a list of clauses.

It will return a list of tuples where the first element is a `DeviceData` object and the second one is a pandas DataFrame.


In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [dd.Project.project_id == PROJECT_ID, dd.Device.pkey == device_pkey],
        limit=1,
        session=session,
    )
    device_data, df = data_tuples[0]  # each tuple has DeviceData and pd.Dataframe
    device_data_pkey = device_data.pkey
    print(device_data)

In [ ]:
device_data_pkey

In [ ]:
df

In [ ]:
plt.plot(df["wavelength"], df["output_power"])
plt.xlabel("wavelength (nm)")
plt.ylabel("output power (dBm)")
plt.title(device_id)

As you have 64 dies per wafer you get one measurement per die.

In [ ]:
len(data_tuples)

You can aggregate the dataframes and the device data objects

You can use the `DeviceData` object to traverse the data model and access additional fields.

You can go from DeviceData to any other tables by following the dashed arrows.

Each column is an attribute on the object representing the table.

![](https://i.imgur.com/CfOHl57.png)

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [dd.Project.project_id == PROJECT_ID, dd.Device.pkey == device_pkey],
        limit=1,
        session=session,
    )
    dfs = [dt[1] for dt in data_tuples]  # dataframes
    dds = [dt[0] for dt in data_tuples]  # device data objects

    print("device id: ", dds[0].device.device_id)
    print("die x: ", dds[0].die.x)
    print("die y: ", dds[0].die.y)
    print("wafer id: ", dds[0].die.wafer.wafer_id)
    print("cell id: ", dds[0].device.cell.cell_id)
    print("parent cell id: ", dds[0].device.parent_cell.cell_id)
    print("project id: ", dds[0].device.cell.project.project_id)
    print(dds[0].device.cell)

For example, you can reach the Cell table and all its columns:

## Build table for JMP

You can build a flat table by combining all the device data and pandas dataframes.  

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.Device.device_id == device_id,
            dd.Die.x == 0,
            dd.Die.y == 0,
        ],
        session=session,
    )
    len(data_tuples)
    dds = [dt[0] for dt in data_tuples]  # device data objects
    dfs = [dt[1] for dt in data_tuples]  # dataframes
dfs[0]

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.Device.device_id == device_id,
            dd.Die.x == 0,
            dd.Die.y == 0,
        ],
        session=session,
    )

    dfs_all = []
    dds = [dt[0] for dt in data_tuples]  # device data objects
    dfs = [dt[1] for dt in data_tuples]  # dataframes

    for device_data, df in zip(dds, dfs, strict=False):
        df["device_id"] = device_data.device.device_id
        df["die_x"] = device_data.die.x
        df["die_y"] = device_data.die.y
        df["wafer_id"] = device_data.die.wafer.wafer_id
        df["cell_id"] = device_data.device.cell.cell_id
        df["parent_cell_id"] = device_data.device.parent_cell.cell_id
        dfs_all.append(df)

In [ ]:
dfs_all = pd.concat(dfs_all)  # You can concatenate all dataFrames together
dfs_all

## Advanced queries


To build advanced queries to filter metadata you can use the `attribute_filter` method. You can also use `or_`, `and_` for conditional clauses.

### Conditional filter

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [dd.Project.project_id == PROJECT_ID, dd.Device.device_id == device_id],
        session=session,
    )
len(data_tuples)

By default anything you pass to the list joins the clauses as `and_`

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.and_(
                dd.Project.project_id == PROJECT_ID, dd.Device.device_id == device_id
            )
        ],
        session=session,
    )
len(data_tuples)

You can also use an OR condition.

The query below filters by project id `PROJECT_ID` AND either  device  id (`ring-20-200` OR `ring-20-150`).

Because there are 21 measurements per device, one for each die, if you look for two specific devices.

In [ ]:
device_id = "rings_RingDouble-20-200-_add_fiber_array_ea36ac06_331580_243771"

with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.or_(dd.Device.device_id == device_id, dd.Device.device_id == device_id),
        ],
        session=session,
    )
len(data_tuples)

You can also combine conditionals.

In the example below, you can get all the data for the specified project id, device id and either die coordinate.

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.Device.device_id == device_id,
            dd.or_(
                dd.and_(dd.Die.x == 1, dd.Die.y == 1),  # either die 1,1
                dd.and_(dd.Die.x == 0, dd.Die.y == 0),  # or die 0,0
            ),
        ],
        session=session,
    )
len(data_tuples)

### Attribute filter

You can filter attributes of any data models by passing a list of conditions as shown below.

You can use `dd.Cell/dd.Wafer/...`, `key`, `value`

You can only filter for values that are str, bool, int, float.

The query below filters Cells with waveguide gap 0.150 that are in a particular project and die coordinates.

In [ ]:
with dd.get_session() as session:
    device_data_objects = dd.get_data_by_query(
        [
            dd.attribute_filter(
                dd.Cell, "gap_um", 0.15
            ),  # checking for values in JSON attributes
            dd.Project.project_id == PROJECT_ID,
            dd.Die.x == 0,
            dd.Die.y == 0,
        ],
        session=session,
    )

In [ ]:
len(device_data_objects)

You can combine conditional and attribute filter clauses together.

You can get the data for Cells for a given project with 150nm 0um and either radius 20um or 10um

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.Die.x == 0,
            dd.Die.y == 0,
            dd.attribute_filter(
                dd.Cell, "gap_um", 0.15
            ),  # checking for values in JSON attributes
            dd.or_(
                dd.attribute_filter(dd.Cell, "radius_um", 20),
                dd.attribute_filter(dd.Cell, "radius_um", 10),
            ),
        ],
        session=session,
    )
len(data_tuples)